In [ ]:
import numpy as np
import skimage
import matplotlib.pyplot as plt
import skimage

from skimage.feature import hog
from skimage import data, exposure
from skimage.exposure import histogram
from skimage.feature import match_template
from skimage.transform import rescale
from skimage.util import random_noise
from scipy.signal import correlate2d
from skimage import io
from skimage.transform import resize

size = 256

cats = [resize(io.imread(f"animals/cats/cats_0000{idx}.jpg"), (size, size)) for idx in range(1,6)]
dogs = [resize(io.imread(f"animals/dogs/dogs_0000{idx}.jpg"), (size, size)) for idx in range(1,6)]
pandas = [resize(io.imread(f"animals/pandas/panda_0000{idx}.jpg"), (size, size)) for idx in range(1,6)]
images = cats + dogs + pandas

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
axes[0].imshow(cats[0]) and axes[0].axis('off')
axes[1].imshow(dogs[0]) and axes[1].axis('off')
axes[1].imshow(pandas[0]) and axes[1].axis('off')

plt.show()

cats_hog = [hog(img, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), channel_axis=2) for img in cats]
dogs_hog = [hog(img, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), channel_axis=2) for img in dogs]
pandas_hog = [hog(img, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), channel_axis=2) for img in pandas]

# étiquettes (1 pour les vélos, 2 pour les voitures)
cats_labels = 1 * np.ones((len(cats_hog), ))
dogs_labels = 2 * np.ones((len(dogs_hog), ))
pandas_labels = 3 * np.ones((len(pandas_hog), ))
